<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

--2020-09-08 12:20:11--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3247 (3.2K) [text/plain]
Saving to: ‘vsc.py’

vsc.py              100%[===================>]   3.17K  --.-KB/s    in 0s      

2020-09-08 12:20:11 (49.1 MB/s) - ‘vsc.py’ saved [3247/3247]

--2020-09-08 12:20:11--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2871 (2.8K) [text/plain]
Saving to: ‘trust_ncg.py’

trust_ncg.py 

In [1]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [2]:
p = che.Props(['Ethanol', 'Isopropanol', 'Water'])

In [3]:
def model(c):
    c=DotMap(c)
    x=che.qtox(c.Vq)
    y=che.qtox(c.Lq)
    z=che.qtox(c.Fq)
    V = x * c.Vtot
    L = y * c.Ltot
    F = z * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix(0., F, c.FT)
    VH = p.Hmix(V, 0., c.flashT)
    LH = p.Hmix(0., L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (x*p.Pvap(c.flashT) - y*c.flashP)
    fugacity = (fugacity/jnp.sum(y*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [4]:
s=DotMap()
s.Ftot=10
s.Fq = che.xtoq([1/3, 1/3, 1/3])
s.FT = 380
s.flashP= 101325

v=DotMap()
v.Vq = che.xtoq([0.4, 0.35, 0.25])
v.Lq = che.xtoq([0.25, 0.35, 0.4])
v.flashT = 300
v.Vtot = s.Ftot/2
v.Ltot = s.Ftot/2

/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
c=vsc.VSC(v,s)
model(c.xtoc(c.x))

["<class 'int'>, 380\n", "<class 'jax.interpreters.xla.DeviceArray'>, [0. 0.]\n", "<class 'int'>, 10\n", "<class 'jax.interpreters.xla.DeviceArray'>, [-0.47000363 -0.13353139]\n", "<class 'float'>, 5.0\n", "<class 'jax.interpreters.xla.DeviceArray'>, [0.47000363 0.33647224]\n", "<class 'float'>, 5.0\n", "<class 'int'>, 101325\n", "<class 'int'>, 300\n"]
["<class 'float'>, nan\n", "<class 'jax.interpreters.xla.DeviceArray'>, [nan nan]\n", "<class 'float'>, nan\n", "<class 'jax.interpreters.xla.DeviceArray'>, [-0.47000363 -0.13353139]\n", "<class 'float'>, 5.0\n", "<class 'jax.interpreters.xla.DeviceArray'>, [0.47000363 0.33647224]\n", "<class 'float'>, 5.0\n", "<class 'float'>, nan\n", "<class 'int'>, 300\n"]


DeviceArray(0.32308156, dtype=float64)

In [6]:
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:2, f: 0.323081563517435
1:3, f: 0.3015989558452608
2:3, f: 0.29402088854662245
3:3, f: 0.2878546352760357
4:5, f: 0.2791400448297653
5:5, f: 0.25240606407786015
6:5, f: 0.2081318611612712
7:4, f: 0.2081318611612712
8:8, f: 0.14723663436738843
9:5, f: 0.14723663436738843
10:2, f: 0.15338836552591653
11:2, f: 0.15338836552591653
12:4, f: 0.14070486983726183
13:3, f: 0.14070486983726183
14:2, f: 0.12535651455110527
15:3, f: 0.11233212403008636
16:4, f: 0.08954008726444519
17:7, f: 0.07207024809728627
18:7, f: 0.06494446419008368
19:7, f: 0.06494446419008368
20:3, f: 0.06494446419008368
21:4, f: 0.06470209530151522
22:3, f: 0.06419681737154025
23:7, f: 0.06192898206416038
24:4, f: 0.06422171879826846
25:2, f: 0.0540641391263168
26:2, f: 0.0540641391263168
27:4, f: 0.040905898542103615
28:3, f: 0.040905898542103615
29:3, f: 0.033710571628278764
30:4, f: 0.027296067785333403
31:2, f: 0.0233288203038306
32:4, f: 0.009065074416898657
33:6, f: 0.004045007462079778
34:7, f: 7.682480715177879e-

In [7]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)
df


,,"(vector2, 1)","(vector2, 2)"
FT,380,,
Fq,,0,0
Ftot,10,,
Lq,,0.0587178,0.050612
Ltot,9.3114,,
Vq,,-0.754227,-0.606328
Vtot,0.688599,,
flashP,101325,,
flashT,361.399,,


,,"(vector2, 1)","(vector2, 2)"
FT,350,,
Fq,,0,0
Ftot,10,,
Lq,,0.401012,0.336411
Ltot,5.28612,,
Vq,,-0.414091,-0.32138
Vtot,4.71388,,
flashP,101325,,
flashT,359.966,,
q,0.5,,
